# multiscale model fitting for Toy2

### start with initalizing many things

In [1]:
# imports
import os
# import sys
import torch
# import pickle
import numpy as np
# import matplotlib.pyplot as plt
# from tqdm.notebook import tqdm
# import time
# import math

# module_path = os.path.abspath(os.path.join('../src/'))
# if module_path not in sys.path:
#     sys.path.append(module_path)
    
    
# import torch_cae_multilevel_V4 as net
# import ResNet as tnet
# import utils

In [7]:
# paths
data_dir = './data/toy2'
model_dir = './model/toy2'
result_dir = './result/toy2'

#load data
train_data = torch.tensor(np.load(os.path.join(data_dir, 'train_data.npy')))
val_data = torch.tensor(np.load(os.path.join(data_dir, 'val_data.npy')))
test_data = torch.tensor(np.load(os.path.join(data_dir, 'test_data.npy')))

data_of_sizes = {}
current_size = 2
# unresolved_dict = {}
# data_levels_dict = {}


In [12]:
#functions, will move these to a utils file eventually 
#====================================================================================
# def data_of_size(data,size):
#     """
#     Takes averages to shrink size of data
#     Takes data of size (n_points, dim, dim) and shrinks to size (n_points, size, size)
#     takes averages to shrink
#     """
#     return decrease_to_size(torch.tensor(data).unsqueeze(1), size)[:,0,:,:]
#====================================================================================


def isPowerOfTwo(n):
    """
    checks if n is a power of two
    
    input: n, int
    
    output: boolean
    """
    return (np.ceil(np.log2(n)) == np.floor(np.log2(n)));
#====================================================================================
def shrink(data, low_dim):
    '''
    Shrinks data to certain size; either averages or takes endpoints
    
    inputs:
        data: array of size (n_points, n_timesteps, dim, dim) that will shrink
        low_dim: int, size to shrink to, low_dim must be less than or equal to dim
        
    output:
        data: array of size (n_points, n_timesteps, low_dim, low_dim)
    '''
    
    #check inputs
    assert len(data.shape) == 4
    n_points, n_timesteps, dim, _ = data.shape
    assert dim >= low_dim
    assert isPowerOfTwo(low_dim)
    
    if dim == low_dim: #same size, no change
        return data
    
    while(dim > low_dim):
        #shrink by 1 level until same size
        data = apply_local_op(data.float(), 'cpu', ave=average)
        current_size = data.shape[-1]
        
    return data
#====================================================================================
def ave_one_level(data):
    '''
    takes averages to shrink data 1 level
    
    inputs:
        data: tensor of size (n_points, n_timesteps, dim, dim) that will shrink
        
    output:
        processed data: tensor of size (n_points, n_timesteps, dim/2, dim/2)
    '''
    device = 'cpu'
    if not torch.is_tensor(data): #needs to be a tensor
        data = torch.tensor(data)
        
    assert len(data.shape)
#     if data.shape != 4:
#         print("data.shape = ", data.shape)
#         print("data.shape should be of length 4")
    n_points, n_timesteps, dim, _ = data.shape
    
    #dim needs to be even 
    assert dim % 2 == 0
    
    data_right_size = torch.flatten(data, 0,1).unsqueeze(1).float()
    
#     n = min(in_channels, out_channels)
    op = torch.nn.Conv2d(1, 1, 2, stride=2, padding=0).to(device)
   
    op.weight.data = torch.zeros(op.weight.data.size()).to(device)
    op.bias.data = torch.zeros(op.bias.data.size()).to(device)
    op.weight.data[0,0, :, :] = torch.ones(op.weight.data[0,0, :, :].size()).to(device) / 4

    # make them non-trainable
    for param in op.parameters():
        param.requires_grad = False
        
    print("Transforming")
        
    shrunk = op(data_right_size)
    
    print("reshape to print")
    
    return shrunk.squeeze(1).reshape((n_points, n_timesteps, dim//2, dim//2))



In [3]:
print(train_data.shape)
print(ave_one_level(train_data).shape)

(100, 500, 128, 128)


NameError: name 'ave_one_level' is not defined

In [4]:

def apply_local_op(data, device, mode='conv', ave=True):
    
    '''
    Shrinks data to certain size; either averages or takes endpoints
    
    inputs:
        data: array of size (n_points, n_timesteps, dim, dim) that will shrink
        low_dim: int, size to shrink to, low_dim must be less than or equal to dim
        average: boolean, True will average, False will just take endpoints
        
    output:
        data: array of size (n_points, n_timesteps, low_dim, low_dim)
    '''
    
    """
    :param data: data to be processed
    :param device: which device is the data placed in?
    :param mode: string, 'conv' or 'deconv'
    :param ave: if to use local average or sample the center
    :return: processed data
    """
    in_channels, out_channels = 1,1#, _, _ = data.size()
#     print("data.size() = ", data.size())
    n = min(in_channels, out_channels)
    if mode == 'conv':
        op = torch.nn.Conv2d(out_channels, out_channels, 2, stride=2, padding=0).to(device)
    elif mode == 'deconv':
        op = torch.nn.ConvTranspose2d(out_channels, out_channels, 3, stride=2, padding=0).to(device)
    else:
        raise ValueError('mode can only be conv or deconv!')
    op.weight.data = torch.zeros(op.weight.data.size()).to(device)
    op.bias.data = torch.zeros(op.bias.data.size()).to(device)

    for i in range(n):
        if mode == 'conv':
            if ave:
                op.weight.data[i, i, :, :] = torch.ones(op.weight.data[i, i, :, :].size()).to(device) / 4
            else:
                op.weight.data[i, i, 1, 1] = torch.ones(op.weight.data[i, i, 1, 1].size()).to(device)
        elif mode == 'deconv':
            op.weight.data[i, i, :, :] = torch.ones(op.weight.data[i, i, :, :].size()).to(device) / 4
            op.weight.data[i, i, 0, 1] += 1 / 4
            op.weight.data[i, i, 1, 0] += 1 / 4
            op.weight.data[i, i, 1, 2] += 1 / 4
            op.weight.data[i, i, 2, 1] += 1 / 4
            op.weight.data[i, i, 1, 1] += 1 / 4
            op.weight.data[i, i, 1, 1] += 1 / 2

    # make them non-trainable
    for param in op.parameters():
        param.requires_grad = False

    return op(data)

In [12]:
print(train_data.shape)
# train_data_tensor = torch.tensor(train_data)
print(torch.tensor(train_data).size())
to_shrink = torch.flatten(train_data_tensor, 0,1).unsqueeze(1)
print(to_shrink.size())

smaller = apply_local_op(to_shrink.float(), 'cpu',)
print( smaller.shape)

(100, 500, 128, 128)
torch.Size([100, 500, 128, 128])
torch.Size([50000, 1, 128, 128])
torch.Size([50000, 1, 64, 64])


In [16]:
grow_again = smaller.squeeze(1).reshape(100,500,64,64)
print(grow_again.shape)

torch.Size([100, 500, 64, 64])


In [13]:
print(train_data.shape)
processed = ave_one_level(train_data)
print(processed.shape)

torch.Size([100, 500, 128, 128])
Transforming
reshape to print
torch.Size([100, 500, 64, 64])
